In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 2019-04-01 First look
Catalina gave me her first results of the SPT. Let's have a look.

## Preliminaries

Here I put a collection of stuff that will allow me to do the analysis more easily.

In [ ]:
def trajectories(data) :
    
    # extract the tracks
    tracks = data.groupby('TRACK_ID')
    trajs = []
    for name, track in tracks :
        try :
            x = track['EDGE_X_LOCATION']
            y = track['EDGE_Y_LOCATION']
        except KeyError :
            x = track['POSITION_X']
            y = track['POSITION_Y']
        trajs.append(np.array([x, y]).T)
    return trajs

In [ ]:
def angle(v1, v2) :
    d = np.dot(v1, v2)
    v = v1[0]*v2[1] - v1[1]*v2[0]
    nv1 = np.linalg.norm(v1)
    nv2 = np.linalg.norm(v2)
    return np.sign(v)*np.arccos(d/(nv1*nv2))

In [ ]:
def angle_analysis(trajectories) :

    # iterate through the trajectories
    angles = []
    for traj in trajectories :
        
        # calculate the difference in x values and y values
        delta_X = np.diff(traj[:, 0], axis=0)[1:]
        delta_Y = np.diff(traj[:, 1], axis=0)[1:]

        # calculate the normalized tangent vectors
        T = np.array([delta_X, delta_Y]).T
        
        # angle analysis
        for i in range(1, len(T)) :
            angles.append(angle(T[i-1], T[i]))
    
    return angles

In [ ]:
class SPT :
    
    def __init__(self, datadir) :
        
        # data directory
        self.datadir = datadir
        self.spots_fname = '%s/Spots in tracks statistics.txt'%(self.datadir)
        self.links_fname = '%s/Links in tracks statistics.csv'%(self.datadir)
        
        # load data
        self.spots = pd.read_csv(self.spots_fname, sep = '\t')
        self.links = pd.read_csv(self.links_fname)
        
        # process tracks
        tracks_spots = self.spots.groupby('TRACK_ID')
        tracks_links = self.links.groupby('TRACK_ID')
        
        # process trajectories
        self.trajectory_spots = trajectories(self.spots)
        self.trajectory_links = trajectories(self.links)
        
        # process displacement
        self.displacement_links = self.links['DISPLACEMENT']
        self.displacement_spots = []
        for traj in self.trajectory_spots :
            self.displacement_spots.extend(np.linalg.norm(np.diff(traj, axis = 0), axis = 1))
        
        # process angles
        self.angles = angle_analysis(self.trajectory_spots)

## Ethanol-treated cell

In [ ]:
# basic parameters
treatment = 'EtOH'
root_datadir = '../data'
cell_id = '016'
cell_n = 'cell2'

# directory name
datadir = '%s/%s_1000frames/%s_%s_%s/10 Spots'%(root_datadir, treatment, treatment, cell_id, cell_n)

# init data
etoh = SPT(datadir)

Let's plot.

In [ ]:
fig = plt.figure(figsize=(10,5))
for traj in etoh.trajectory_spots :
    plt.plot(traj[:, 0], traj[:, 1])
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5))
for traj in etoh.trajectory_links :
    plt.plot(traj[:, 0], traj[:, 1])
plt.show()

You can clearly see the shape of the nucleus here.

Let's see if we're doing things correctly by calculating the displacement of the particles.

In [ ]:
plt.hist(etoh.displacement_spots, bins = 100)
plt.xlabel('Displacement', fontsize = 18)
plt.ylabel('Frequency', fontsize = 18)
plt.show()

This plot would be exactly equal in shape as the distribution of the velocities, because the sampling is done at 
fixed time intervals.

## R5020

In [ ]:
# basic parameters
treatment = 'R5020'
root_datadir = '../data'
cell_id = '004'
cell_n = 'cell1'

# build file name
datadir = '%s/%s_1000frames/%s_%s_%s/10 spots'%(root_datadir, treatment, treatment, cell_id, cell_n)
R5020 = SPT(datadir)

In [ ]:
fig = plt.figure(figsize=(10,5))
for traj in R5020.trajectory_spots :
    plt.plot(traj[:, 0], traj[:, 1])
plt.show()

In [ ]:
plt.hist(R5020.displacement_links, bins = 100)
plt.xlabel('Displacement', fontsize = 18)
plt.ylabel('Frequency', fontsize = 18)
plt.show()

Okay, pretty significant difference already. Let's plot the two distributions together.

In [ ]:
plt.hist(etoh.displacement_spots, bins = 100, alpha = 0.5, color = 'b', label = 'EtOH')
plt.hist(R5020.displacement_links, bins = 100, alpha = 0.5, color = 'r', label = 'R5020')
plt.xlabel('Displacement', fontsize = 18)
plt.ylabel('Frequency', fontsize = 18)
plt.legend(loc='upper right')
plt.show()

## Angle distribution
An interesting question concerns the distribution of the angles in the trajectories.

In [ ]:
def plot_angles(angles, bins, title) :
    counts, angle_edges = np.histogram(angles, bins = bins)
    angle_centers = angle_edges[1:] - np.ediff1d(angle_edges)
    ax = plt.subplot(111, projection = 'polar')
    bars = ax.bar(angle_centers, counts, width = 2*np.pi/bins, edgecolor = 'k')
    ax.set_title(title, y = 1.1, fontsize = 18)
    plt.show()

In [ ]:
plot_angles(etoh.angles, 24, 'EtOH')
plot_angles(R5020.angles, 24, 'R5020')

This concludes the analysis of two single files.

The next step is to do the analysis of all the tracks.

In [ ]:
for name, track in R5020.links.groupby('TRACK_ID') :
    velocity = track['VELOCITY']
    if len(velocity) > 40 :
        plt.plot(range(len(velocity)), velocity)

In [ ]:
plt.hist(etoh.links['VELOCITY'], bins = 100, alpha=0.5)
plt.hist(R5020.links['VELOCITY'], bins = 100, alpha=0.5)
plt.show()